# Laboratorio 7 

##### Manuel Archila 161250
##### Diego Franco 20240
##### Juan Diego Avila 20090

## Task 1 - Práctica

### Ejercicio 1

In [1]:
import torch

print(torch.cuda.is_available())

True


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

# Definición del modelo
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.tanh(F.avg_pool2d(self.conv1(x), 2))
        x = torch.tanh(F.avg_pool2d(self.conv2(x), 2))
        x = x.view(-1, 16*5*5)
        x = torch.sigmoid(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

# Hiperparámetros
BATCH_SIZE = 64
EPOCHS = 15
LR = 0.01

# Transformaciones y carga del dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Instancia del modelo, función de pérdida y optimizador
model = LeNet5().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LR)

# Entrenamiento
for epoch in range(EPOCHS):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx*len(data)}/{len(train_loader.dataset)} "
                  f"({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

# Evaluación
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item() 
        pred = output.argmax(dim=1, keepdim=True) 
        correct += pred.eq(target.view_as(pred)).sum().item()


print(f"({100. * correct / len(test_loader.dataset):.0f}%)\n")


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.290354
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.315570
Train Epoch: 0 [12800/60000 (21%)]	Loss: 2.299078
Train Epoch: 0 [19200/60000 (32%)]	Loss: 2.298903
Train Epoch: 0 [25600/60000 (43%)]	Loss: 2.300027
Train Epoch: 0 [32000/60000 (53%)]	Loss: 2.312724
Train Epoch: 0 [38400/60000 (64%)]	Loss: 2.307347
Train Epoch: 0 [44800/60000 (75%)]	Loss: 2.314235
Train Epoch: 0 [51200/60000 (85%)]	Loss: 2.299103
Train Epoch: 0 [57600/60000 (96%)]	Loss: 2.297892
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300854
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.304880
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.295555
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.306973
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.288721
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.300674
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.300735
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.296987
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.297271
Train Epoch: 1 [57600/60000 (96%)]	Loss: 2.303181
Train Epoch:

### Ejercicio 2

In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F

class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        
        # Capa convolucional 1
        self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2)
        self.relu1 = nn.ReLU(inplace=True)
        self.lrn1 = nn.LocalResponseNorm(5, alpha=1e-4, beta=0.75, k=2)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2)
        
        # Capa convolucional 2
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU(inplace=True)
        self.lrn2 = nn.LocalResponseNorm(5, alpha=1e-4, beta=0.75, k=2)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2)
        
        # Capa convolucional 3
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU(inplace=True)
        
        # Capa convolucional 4
        self.conv4 = nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU(inplace=True)
        
        # Capa convolucional 5
        self.conv5 = nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1)
        self.relu5 = nn.ReLU(inplace=True)
        self.pool5 = nn.MaxPool2d(kernel_size=3, stride=2)
        
        # Capas completamente conectadas
        self.fc6 = nn.Linear(256 * 6 * 6, 4096)
        self.relu6 = nn.ReLU(inplace=True)
        self.dropout6 = nn.Dropout(0.5)
        
        self.fc7 = nn.Linear(4096, 4096)
        self.relu7 = nn.ReLU(inplace=True)
        self.dropout7 = nn.Dropout(0.5)
        
        self.fc8 = nn.Linear(4096, num_classes)
        
    def forward(self, x):
        x = self.pool1(self.lrn1(self.relu1(self.conv1(x))))
        x = self.pool2(self.lrn2(self.relu2(self.conv2(x))))
        x = self.relu3(self.conv3(x))
        x = self.relu4(self.conv4(x))
        x = self.pool5(self.relu5(self.conv5(x)))
        
        x = x.view(x.size(0), 256 * 6 * 6)
        
        x = self.dropout6(self.relu6(self.fc6(x)))
        x = self.dropout7(self.relu7(self.fc7(x)))
        
        x = self.fc8(x)
        
        return F.softmax(x, dim=1)

# Función para entrenar el modelo
def train_model(model, train_loader, criterion, optimizer, num_epochs=30):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Función para evaluar el modelo
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f"Accuracy on test set: {accuracy}%")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

model = AlexNet(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

train_model(model, train_loader, criterion, optimizer, num_epochs=30)

evaluate_model(model, test_loader)

Files already downloaded and verified
Files already downloaded and verified
Epoch 1/30, Loss: 2.302589490895381
Epoch 2/30, Loss: 2.3025704125309234
Epoch 3/30, Loss: 2.3025254858729176
Epoch 4/30, Loss: 2.302226912944823
Epoch 5/30, Loss: 2.25119497983352
Epoch 6/30, Loss: 2.213179388619445
Epoch 7/30, Loss: 2.1572559105465783
Epoch 8/30, Loss: 2.111590341533846
Epoch 9/30, Loss: 2.071355868635885
Epoch 10/30, Loss: 2.041391796163281
Epoch 11/30, Loss: 1.9850449238896675
Epoch 12/30, Loss: 1.9442216006996076
Epoch 13/30, Loss: 1.9191369117068513
Epoch 14/30, Loss: 1.886217120815726
Epoch 15/30, Loss: 1.8580890584479817
Epoch 16/30, Loss: 1.833253474948961
Epoch 17/30, Loss: 1.8073494653872517
Epoch 18/30, Loss: 1.7937313595696178
Epoch 19/30, Loss: 1.7717730893808252
Epoch 20/30, Loss: 1.7616859060114303
Epoch 21/30, Loss: 1.758428267048448
Epoch 22/30, Loss: 1.7453365409770585
Epoch 23/30, Loss: 1.732283755031693
Epoch 24/30, Loss: 1.710803150216027
Epoch 25/30, Loss: 1.7055367705462

a. ¿Cuál es la diferencia principal entre ambas arquitecturas?

- LaNet-5: es una arquitectura más simple, contando con 2 capas convolucionales y 3 capas completamente conectadas. Se utiliza la función de ctiación de tanh y existe un menor número de parámetros y menor profundidad que logra AlexNet.
- AlexNet: es una arquitectura más compleja, que posee 5 capas convolucionales y 3 capas completamente conectadas. Utiliza la función de activación ReLU e implementa funciones como dropout para evitar el sobreajuste. Permite mayor profundiad y tiene un mayor número de parámetros que LaNet-5.

b. Podría usarse LeNet-5 para un problema como el que resolvió usando AlexNet? ¿Y viceversa?
- Tecnicamente LeNet-5 si se podría usar para problemas que resolvio Alexnet. Sin embargo LeNet-5 tiene un menor capaciddad y profundidad haciendo que no funcione tan bien como Alexnet. Por otro lado, Alexnet si se podría usar para problemas que resolvio LeNet-5, pero al tener una mayor capacidad y profundidad. Esta mayor capacidad puede hace que se use un modelo más complejo para problrmas más sencillos, lo cual puede llevar a un uso innecesario de recursos.

c. Indique claramente qué le pareció más interesante de cada arquitectura
- Lo que más nos llamó la antención de LeNet-5 es la simplicidad del modelo. A pesar de ser considerablemente simple, logró obtener resultados bastante buenos. Por otro lado, lo que más nos llamó la atención de Alexnet es la complejidad del modelo. A pesar de ser un modelo complejo, no logró resultados tan buenos como LeNet-5. Esto nos hace pensar que la complejidad de un modelo no necesariamente se traduce en mejores resultados.

Investigue e indique en qué casos son útiles las siguientes arquitecturas, agregue imagenes si esto le ayuda a una mejor comprensión

a. GoogleNet (Inception)

- GoogleNet, también conocida como Inception, es una arquitectura de CNN desarrollada por Google. Se destacó por su profundidad y eficiencia en la utilización de los recursos.
- Es útil en casos donde se requieren redes profundas pero se desea mantener un uso eficiente de los recursos computacionales. GoogleNet utiliza una estructura llamada "módulos Inception" que combina múltiples tamaños de filtros de convolución en paralelo, permitiendo la extracción de características a diferentes escalas.
- Útil para tareas de clasificación de imágenes, detección de objetos y segmentación semántica.

b. DenseNet (Densely Connected Convolutional Networks)

- DenseNet es una arquitectura de CNN que se caracteriza por su densa conectividad entre capas. Cada capa está conectada directamente con todas las capas subsiguientes.
- Es útil en casos donde se desea un mejor flujo de información y gradientes más fuertes a lo largo de la red, lo que facilita el entrenamiento de redes profundas.
- Útil para tareas de clasificación de imágenes, detección de objetos y segmentación semántica.

c. MobileNet

- MobileNet es una arquitectura de CNN diseñada para aplicaciones en dispositivos móviles y embebidos con recursos computacionales limitados.
- Es útil en casos donde se necesita una red ligera y rápida, como en aplicaciones de visión por computadora en dispositivos móviles.
- Útil para tareas de clasificación de imágenes, detección de objetos en tiempo real y otras aplicaciones de visión en dispositivos móviles.


d. EfficientNet

- EfficientNet es una familia de arquitecturas de CNN que buscan optimizar el equilibrio entre el rendimiento y la eficiencia computacional mediante el uso de escalado compuesto.
- Es útil en casos donde se desean modelos con un buen rendimiento pero que sean escalables en términos de tamaño y requisitos computacionales.
- Útil para una variedad de tareas de visión por computadora, desde clasificación de imágenes hasta detección de objetos y segmentación semántica.

¿Cómo la arquitectura de transformers puede ser usada para image recognition?

La arquitectura de Transformers se puede usar en el reconocimiento de imágenes al tratar las imágenes como secuencias de parches y aplicar mecanismos de atención y transformación para capturar información espacial y contextual. Esto se puede hacer al obtener las características, el uso de atención multi-cabeza y agregar información. Los modelos de visión Transformer se entrenan en conjuntos de datos etiquetados, se ajustan finamente en tareas específicas y permiten la transferencia de aprendizaje.